# A first classifier in pytorch : language identification using a bag of word input  


The basic way of creating a neural network in Pytorch is to define a class inheriting from the nn.Module.

The steps are:
- define a class inheriting from the nn.Module, which should contain at least declarations (members) for all that will be parameters of the network (=to be learnt)
- define a forward method
- you don't have to define the backpropagation method ! given the forward method, pytorch automatically and dynamically builds the computation graphs and computes gradients when necessary
- pytorch also allows you to easily switch the computing device : from CPU to GPU


## NB: About the batch axis

**NB** all forward propagations in nn.Module and subclasses take as input a (mini-)batch of input vectors, and not a single vector. Remember the first axis (axis 0) is **always** for the batch.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim # for the optimizers (SGD, Adam ...)


Creating a MLP for language identification in PyTorch
======================================







## Defining the network

The objective is to build a classifier for language identification (input = sequence, output=its language), representing the input as a BOW vector.

We will build a MLP with a single hidden layer with a non-linear activation function, and softmax to get probabilities of classes.
We'll then use the negative log likelihood loss, and stochastic gradient descent to learn the parameters.

Because log-likelihood supposes to take the log of probabilities, in our network we will directly output log-softmax instead of softmax.



In [2]:
class MyMLP(nn.Module):  # inheriting from nn.Module

    def __init__(self, nb_classes, d, hidden_layer_size):
        """ - d is the size of the vectors representing the objects to classify
            - hidden_layer_size is ... the size of the hidden layer
        """
        # calls the init function of the superclass, i.e. nn.Module
        super(MyMLP, self).__init__()

        # The parameters of the network that must be learnt
        # are inferred thanks to pytorch machinary,
        # from what is declared in the constructor

        # linear layer Wx + b
        # from size d to hidden_layer_size
        # => implicitely contains parameters W and b
        # cf. https://pytorch.org/docs/stable/generated/torch.nn.Linear.html
        # which are randomly initialized by default
        self.linear_1 = nn.Linear(d, hidden_layer_size)

        # another pair of W and b parameters
        self.linear_2 = nn.Linear(hidden_layer_size, nb_classes)

        # NB: the non-linear activation functions do not have any parameter
        # so we don't need to declare it in the __init__ method
        # they will be used directly in the forward method

    def forward(self, X):
        """
        Input : X batch of inputs : ( shape (b, d) )
                with b is the size of the batch (it does not have to be constant)
        Output : log probabilities (shape (b, nb_classes) )
        """
        # linear combination from input to hidden layer
        out = self.linear_1(X) # out shape (b, hidden_layer_size)

        # activation function at hidden layer
        out = torch.relu(out) # same shape

        # linear combination from hidden to output layer
        # from hidden_layer_size to nb of classes
        out = self.linear_2(out) # out shape (b, nb_classes)

        # Transformation into log-probabilities
        #  Note : Since we're going to use the NLL loss,
        #         we will need to apply log to softmax anyway
        #         we do it here because directly computing log of softmax is numerically more stable
        # CAUTION: the dim=1 is crucial here to tell pytorch on which axis apply the softmax
        return F.log_softmax(out, dim=1) # out shape (b, nb_classes)

# examples
my_classifier = MyMLP(nb_classes=3, d=20, hidden_layer_size=10)

# nn.Module and any subclass (e.g. nn.Linear, BoWClassifier)
# records what are the parameters
# depending on what has been declared in the __init__ method.

# For nn.Linear, the first param is matrix W, the second is bias b.
# your module (in this case, MyMLP) will store knowledge of the
# linear_1 and linear_2 members declared in the __init__

# and the parameters have requires_grad set to True of course
for name, param in my_classifier.named_parameters():
    print("PARAM named %s, of shape %s" % (name, str(param.shape)))
    print(param)





PARAM named linear_1.weight, of shape torch.Size([10, 20])
Parameter containing:
tensor([[-0.0944,  0.1588, -0.0088,  0.0451,  0.1925, -0.1704,  0.1946, -0.1108,
         -0.0671,  0.0975,  0.0139,  0.1271, -0.1312, -0.0269,  0.0418, -0.0204,
         -0.0760, -0.0512, -0.1645,  0.2120],
        [ 0.1863, -0.0818, -0.0350,  0.1672,  0.1632,  0.0802,  0.1802,  0.1053,
         -0.1181, -0.1919,  0.0303, -0.0538,  0.1739,  0.0108,  0.0205, -0.0459,
          0.0337,  0.1456,  0.1463,  0.0776],
        [ 0.2032, -0.2037,  0.0703, -0.2075,  0.1451,  0.1667,  0.1935, -0.0595,
          0.0574, -0.0333, -0.0414, -0.1333, -0.0040,  0.0583, -0.0826, -0.2021,
         -0.0144,  0.1294,  0.0820, -0.2167],
        [ 0.1650, -0.0513, -0.2129, -0.0150,  0.0472, -0.1279, -0.1246, -0.1848,
         -0.1692,  0.0551,  0.1989, -0.0976,  0.0237, -0.2154,  0.1625,  0.0228,
          0.0750, -0.0262,  0.2095, -0.1488],
        [ 0.1483, -0.1126,  0.0512, -0.0250,  0.1453,  0.1443, -0.0409,  0.1011,
      

## Toy dataset for language identification

The task we consider is **language identification**,
and we are using a toy data set of 4 training examples, and 2 test examples (below).



In [3]:
train_examples = [("oggi non siamo in ferie".split(), "ITALIAN"),
                  ("give it to me".split(), "ENGLISH"),
                  ("le mie ferie sono state molto belle".split(), "ITALIAN"),
                  ("credo che le ferie sono sempre troppo corte".split(), "ITALIAN"),
                  ("it is a good idea not to get lost".split(), "ENGLISH"),
                  ("give me a good idea please".split(), "ENGLISH"),
                  ("non credo che sia una buona idea".split(), "ITALIAN"),
                  ("no it is not a good idea to get lost at sea".split(), "ENGLISH")]

test_examples = [("io credo che si".split(), "ITALIAN"),
                 ("it is lost on me".split(), "ENGLISH")]




## TODO 1: Encoding data

You need to build a mapping from words to indices from 0 to vocab_size -1
and a mapping from languages (target classes) to indices.

Then encode the n input texts into BOW vectors,
in the form of a torch tensor of shape (n, vocab_size).

**NB**: the vocabulary is that of the training texts. The words in test that are unknown in train will be ignored.

**NB**: you can implement it yourself or use the CountVectorizer of sklearn.

**NB**: if you use the CountVectorizer, you need to convert the obtained numpy sparse matrix into torch Tensor: if A is the numpy matrix, use `torch.from_numpy(A).float()`

In [4]:
# word to index
# and label to index correspondances
w2i = {}
i2w = []
label2i = {}
i2label = []

# TODO:

# fill in the w2i, i2w, label2i and i2label
# using the training examples only

# or use the CountVectorizer of sklearn


VOCAB_SIZE = len(w2i)
NB_CLASSES = len(label2i)

print("VOCAB SIZE:", VOCAB_SIZE)
print("NB CLASSES:", NB_CLASSES)
print("w2i:", w2i)
print("i2w:", i2w)
print("label2i:", label2i)


def convert_examples_to_tensors(examples, w2i, label2i):
    """ Input =
          - list of n examples
             -- each example is a pair [sentence, class label]
             -- a sentence being a list of tokens
          - dictionary of words to indices
          - dictionary of class labels to indices
        Output =
          - X = BOW vectors for the n examples:
              = pytorch tensor of shape ( n , vocabulary size )
                X[i, j] = number of occ of word j in sentence i
          - Y = tensor of shape  ( n ) for indices of gold labels
    """
    # separating input sentences and gold labels
    (sentences, gold_labels) = list(zip(*examples))

    n = len(examples)

    # TODO

    # X = ...
    # Y = ... (use torch.LongTensor to convert a list of class ids to a tensor)
    return X, Y


X_test = None
X_train = None
Y_test = None
Y_train = None

# uncomment when done
#X_train, Y_train = convert_examples_to_tensors(train_examples, w2i, label2i)
#X_test, Y_test = convert_examples_to_tensors(test_examples, w2i, label2i)

#print("X_test", X_test)
#print("Y_test", Y_test)


VOCAB SIZE: 0
NB CLASSES: 0
w2i: {}
i2w: []
label2i: {}


## Use TensorDataset and DataLoader for easy batching

In [5]:
from torch.utils.data import TensorDataset, Dataset, DataLoader


train_dataset = TensorDataset(X_train, Y_train)
test_dataset = TensorDataset(X_test, Y_test)


BATCH_SIZE = 2

# shuffle = True will shuffle the data after each full iteration over the dataset
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE)



AttributeError: 'NoneType' object has no attribute 'size'

In [6]:
my_classifier = MyMLP(nb_classes=NB_CLASSES, d=VOCAB_SIZE, hidden_layer_size=10)

/home/etal2025/.local/lib/python3.11/site-packages/torch/nn/init.py:582: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


## Test of forward propagation (with random parameters)

**NB** : instead of calling the forward method, the pytorch machinary requires to run the __call__ method, callable using the name of the instance

In [7]:
# Test of forward propagation with the randomly initialized parameters:

def run_and_eval(dataloader):
    # NB: when we are not training, we don't have to compute gradients
    # => to be more efficient, we use torch.no_grad()
    with torch.no_grad():
        nb_correct = 0
        for (X, gold_labels) in dataloader:
            log_probs = my_classifier(X)
            # get the highest scoring class for each element in the batch
            pred_labels = torch.argmax(log_probs, dim=1)
            nb_correct += torch.sum( (pred_labels == gold_labels).int()).item()
        accuracy = nb_correct / len(test_dataloader.dataset)
        print("Accuracy:", accuracy)
        return accuracy

run_and_eval(test_dataloader)


NameError: name 'test_dataloader' is not defined

## Training

Loss functions are provided in the nn package.

nn.NLLLoss() is the negative log likelihood loss

Optimization functions are in torch.optim.
Here, we will just use SGD.


In [8]:


# --------- the loss -------------
# negative log likelihood loss
# TODO: check its input and output https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html
loss_function = nn.NLLLoss()

# --------- the optimizer --------
# simplest one: stochastic gradient descent
# we declare the parameters we wish to optimize
# => here we want to optimize all the parameters of our MyMLP instance
optimizer = optim.SGD(my_classifier.parameters(), lr=0.1)

print("\nTraining on:")
for x in train_examples:
    print(x)
print("\n")
print(label2i)


# loop on epochs
for epoch in range(10):
    print("Epoch", epoch)

    for (X, gold_labels) in train_dataloader:

        # Step 1:
        # (re)sets all parameter gradients to 0
        #  before using each batch of inputs
        my_classifier.zero_grad()

        # Step 2: forward propagation
        #         NB: my_classifier(X) implicitely calls my_classifier.forward(X)
        log_probs = my_classifier(X)
        print("LOG PROBS on train at epoch %i:\n" %epoch, log_probs)
        # optional: accuracy on this batch
        pred_labels = torch.argmax(log_probs, dim=1)
        print("PREDICTED LABELS ON BATCH", [ i2label[l] for l in pred_labels ])
        print("     GOLD LABELS ON BATCH", [ i2label[l] for l in gold_labels ])

        # Step 3: Compute the loss (NB: this is the loss for the full batch of inputs X)
        #         The input to the loss function is for each example in X,
        #          the log_probabilities for each class, and the gold label
        loss = loss_function(log_probs, gold_labels)


        # Step 4: Compute the gradients
        loss.backward() # partial derivatives of loss with respect to
                        # all the tensors that - were used to compute loss,
                        #                      - and have requires_grad=True
                        # after this call, all the parameters have their .grad attribute
                        # filled with the partial derivative

        # Step 5: Update the parameters
        #         NB: the optimizer instance knows what are the parameters to update
        optimizer.step()




Training on:
(['oggi', 'non', 'siamo', 'in', 'ferie'], 'ITALIAN')
(['give', 'it', 'to', 'me'], 'ENGLISH')
(['le', 'mie', 'ferie', 'sono', 'state', 'molto', 'belle'], 'ITALIAN')
(['credo', 'che', 'le', 'ferie', 'sono', 'sempre', 'troppo', 'corte'], 'ITALIAN')
(['it', 'is', 'a', 'good', 'idea', 'not', 'to', 'get', 'lost'], 'ENGLISH')
(['give', 'me', 'a', 'good', 'idea', 'please'], 'ENGLISH')
(['non', 'credo', 'che', 'sia', 'una', 'buona', 'idea'], 'ITALIAN')
(['no', 'it', 'is', 'not', 'a', 'good', 'idea', 'to', 'get', 'lost', 'at', 'sea'], 'ENGLISH')


{}
Epoch 0


NameError: name 'train_dataloader' is not defined

In [ ]:
acc = run_and_eval(test_dataloader)
print("accuracy on test after training: %3.2f" % (acc*100))